# RNN Encoder-Decoder

This is the notebook for constructing Theano RNN layers and RNN sentiment classifier. The text for an encoder-decoder can be found in a lot of places, and one reference is here: [http://anie.me/rnn-encoder-decoder/](http://anie.me/rnn-encoder-decoder/). This notebook might also contain a section on vanilla RNN classification task, and a section on RNN autoencoder as well. Note in this implementation, we are being a lot more efficient than some blog posts (like in WildML).

## Loading Data SNLI

We use SNLI for inference task (the main purpose of encoder-decoder). We use a partial IMDB dataset provided by Yoon Kim (also preprocessed into numpy form).